# Homework 2

In [48]:
from sklearn.datasets import load_boston
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score

from scipy import stats
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

X , y = load_boston(return_X_y=True)
df = pd.DataFrame(X,columns = load_boston().feature_names)
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
dtypes: float64(13)
memory usage: 51.5 KB


In [50]:
df.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000
75%,3.677083,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000


## Ön işleme adımları

In [51]:
#boş değer var mı kontrolü
df.isna().sum()

CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
dtype: int64

In [52]:
#aykırı değer var mı kontrolü öncelikle belirli bir skala değeri uyguladık
z=np.abs(stats.zscore(df))
z

array([[0.41978194, 0.28482986, 1.2879095 , ..., 1.45900038, 0.44105193,
        1.0755623 ],
       [0.41733926, 0.48772236, 0.59338101, ..., 0.30309415, 0.44105193,
        0.49243937],
       [0.41734159, 0.48772236, 0.59338101, ..., 0.30309415, 0.39642699,
        1.2087274 ],
       ...,
       [0.41344658, 0.48772236, 0.11573841, ..., 1.17646583, 0.44105193,
        0.98304761],
       [0.40776407, 0.48772236, 0.11573841, ..., 1.17646583, 0.4032249 ,
        0.86530163],
       [0.41500016, 0.48772236, 0.11573841, ..., 1.17646583, 0.44105193,
        0.66905833]])

In [53]:
outliers = list(set(np.where(z > 3)[0]))

len(outliers)

91

In [54]:
# tespit ettiğimiz aykırı değerleri siliyoruz
new_df = df.drop(outliers,axis = 0).reset_index(drop = False)
display(new_df)

y_new = y[list(new_df["index"])]
len(y_new)

,index,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
410,501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
411,502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
412,503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
413,504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


415

In [55]:
#Scaling
X_new = new_df.drop('index', axis = 1)

from sklearn.preprocessing import StandardScaler, MinMaxScaler
X_scaled = MinMaxScaler().fit_transform(X_new)
X_scaled

array([[0.00000000e+00, 2.25000000e-01, 6.78152493e-02, ...,
        3.13953488e-01, 1.00000000e+00, 1.07402512e-01],
       [7.32648551e-04, 0.00000000e+00, 2.42302053e-01, ...,
        6.04651163e-01, 1.00000000e+00, 2.44877726e-01],
       [7.31950458e-04, 0.00000000e+00, 2.42302053e-01, ...,
        6.04651163e-01, 9.87015473e-01, 7.60079313e-02],
       ...,
       [1.90020901e-03, 0.00000000e+00, 4.20454545e-01, ...,
        9.76744186e-01, 1.00000000e+00, 1.29213483e-01],
       [3.60460295e-03, 0.00000000e+00, 4.20454545e-01, ...,
        9.76744186e-01, 9.88993460e-01, 1.56972902e-01],
       [1.43423197e-03, 0.00000000e+00, 4.20454545e-01, ...,
        9.76744186e-01, 1.00000000e+00, 2.03238599e-01]])

In [56]:
X_train, X_test, y_train,y_test = train_test_split(X_scaled,y_new, test_size = 0.30, random_state = 102)

In [57]:
#Fit simple linear model and find coefficients
regression_model = LinearRegression()
regression_model.fit(X_train, y_train)

print(f'Regression model coef: {regression_model.coef_}')

Regression model coef: [-3.52400516e+00 -3.37937090e-01 -3.75375735e-01 -8.88178420e-16
 -4.83118366e+00  1.91265806e+01 -1.09229411e+00 -8.65061590e+00
  4.90505531e+00 -5.06198680e+00 -8.41892795e+00  2.55334063e+00
 -1.46516981e+01]


## Regularized Ridge Regression

In [58]:
#alpha factor here is lambda (penalty term) which helps to reduce the magnitude of coeff

ridge_model = Ridge(alpha = 10)
ridge_model.fit(X_train, y_train)

print(f'Ridge model coef: {ridge_model.coef_}')
#As the data has 10 columns hence 10 coefficients appear here

Ridge model coef: [-1.74042306  0.48148198 -2.32562831  0.         -1.73383444  9.62611295
 -0.88203563 -3.05010214  0.69056119 -2.34213712 -5.73913993  1.56322555
 -9.52336062]


## Regularized Lasso Regression

In [59]:
#alpha factor here is lambda (penalty term) which helps to reduce the magnitude of coeff

lasso_model = Lasso(alpha = 0.2)
lasso_model.fit(X_train, y_train)

print(f'Lasso model coef: {lasso_model.coef_}')
#As the data has 10 columns hence 10 coefficients appear here

Lasso model coef: [ -0.          -0.          -0.           0.          -0.
  13.74589024  -0.          -0.          -0.          -0.69465767
  -5.28938972   0.         -14.66892788]


## Score Comparison

In [60]:
#Model score - r^2 or coeff of determinant
#r^2 = 1-(RSS/TSS) = Regression error/TSS 


#Simple Linear Model
print("Simple Train: ", regression_model.score(X_train, y_train))
print("Simple Test: ", regression_model.score(X_test, y_test))
print('*************************')
#Lasso
print("Lasso Train: ", lasso_model.score(X_train, y_train))
print("Lasso Test: ", lasso_model.score(X_test, y_test))
print('*************************')
#Ridge
print("Ridge Train: ", ridge_model.score(X_train, y_train))
print("Ridge Test: ", ridge_model.score(X_test, y_test))

Simple Train:  0.7268385189778708
Simple Test:  0.774589024610258
*************************
Lasso Train:  0.6621891919904473
Lasso Test:  0.7065682822953839
*************************
Ridge Train:  0.6256395994392034
Ridge Test:  0.6810491832452332
